In [ ]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pecanpy

In [ ]:
%matplotlib inline
plt.style.use("ggplot")

In [ ]:
pd.__version__

In [ ]:
# define user credentials
USER_NAME = ???
PASSWORD = ???

# define db server params
SCHEMA = ???
HOST = ???
PORT = ???
DB = ???

# create the engine that connects to the database...
engine = pecanpy.create_engine(USER_NAME, PASSWORD, HOST, PORT, DB)

In [ ]:
with engine.connect() as con:
    metadata_df = pecanpy.read_metadata_table(con, schema=SCHEMA)

In [ ]:
metadata_df.head()

In [ ]:
time = datetime.datetime.now()
seed = time.hour * 10000 + time.minute * 100 + time.second
prng = np.random.RandomState(seed)
with_egauge_data = metadata_df[metadata_df.egauge_min_time.notnull()]
random_household = with_egauge_data.sample(n=1, random_state=prng)

In [ ]:
# how much available data possibly exists?
start_time = random_household.iloc[0].egauge_min_time
end_time = random_household.iloc[0].egauge_max_time
time_delta = end_time - start_time

In [ ]:
time_delta

In [ ]:
with engine.connect() as con:

    # extract the required parameters
    dataid = random_household.iloc[0].name
    start_time = random_household.iloc[0].egauge_min_time
    end_time = random_household.iloc[0].egauge_max_time

    # pull down data for different aggregations
    minutes_df = pecanpy.read_electricity_egauge_query(con, SCHEMA, dataid, start_time, end_time, "all", 'T')
    qtr_hour_df = pecanpy.read_electricity_egauge_query(con, SCHEMA, dataid, start_time, end_time, "all", "15T")
    hours_df = pecanpy.read_electricity_egauge_query(con, SCHEMA, dataid, start_time, end_time, "all", 'H')

## Minutes data

In [ ]:
minutes_df.shape

In [ ]:
minutes_df.head()

In [ ]:
minutes_df.tail()

In [ ]:
# if gen is not null, then household is a prosumer!
minutes_df[["grid", "use", "gen"]].head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# plot total daily electricity usage
minutes_df.use.resample("D").sum().plot(ax=ax, label="use")
ax.set_title("Electricity use for household {}".format(dataid))
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
hourly_load_curves = (minutes_df.resample("H")
                                .sum()
                                .groupby(lambda timestamp: timestamp.hour)
                                .mean())
hourly_load_curves.use.plot(ax=ax);

## 15-minute data

In [ ]:
qtr_hour_df.head()

In [ ]:
qtr_hour_df.tail()

In [ ]:
# can recover the 15-minute data ay aggregating the minutes data!
minutes_df.resample("15T").mean().tail()

## Hours data

In [ ]:
hours_df.head()

In [ ]:
hours_df.tail()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.set_xlabel("Hour")
ax.set_ylabel("Electricity usage")
ax.set_title("Hourly average usage for household {}".format(dataid))
hours_df.use.groupby(lambda idx: idx.hour).mean().plot(ax=ax);
